# Checking init scale

Our initialization method is somewhat non-standard (due to my mistake):

```python
model.add(
        layers.Dense(
            hidden_size,
            activation=activation,
            kernel_initializer=VarianceScaling(scale=init_scale, mode="fan_out"), # this
            bias_initializer=VarianceScaling(scale=init_scale, mode="fan_out"), # this
        )
    )
    model.add(
        layers.Dense(
            dataset.n_classes if classification else 1,
            kernel_initializer=VarianceScaling(scale=init_scale, mode="fan_in"), # this
            bias_initializer=VarianceScaling(scale=init_scale, mode="fan_in"), # this
            activation=None,
        )
    )
```

This is to check whether it has a significant effect on performance, namely, what happens if we comment out the lines annotated with `# this`. The results of such training are in `0402_check_init_scale`.

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

ms1 = pd.read_feather("0326_mnist_binary/measures.feather")
ms1["init_type"] = "ours"
ms2 = pd.read_feather("0402_check_init_scale/measures.feather")
ms2["init_type"] = "standard"

# ms = pd.concat([ms1, ms2], sort=False)
# ms = ms.reset_index(drop=True)

# print("Removing {} entries".format(sum(ms["gradient_norm_test"].isna())))
ms1 = ms1[~ms1["gradient_norm_test"].isna()]
ms2 = ms2[~ms2["gradient_norm_test"].isna()]
# ms["model.weights_product_reg_coef"] = ms["model.weights_product_reg_coef"].fillna(value=0)

smooth.analysis.remove_constant_columns(ms1, verbose=True)
smooth.analysis.remove_constant_columns(ms2, verbose=True)

ms1 = ms1[
    (ms1["model.hidden_size"] == 256) &
    (ms1["model.gradient_norm_reg_coef"] == 0) &
    (ms1["model.weights_product_reg_coef"] == 0)
]

ms1 = ms1.set_index("dataset.name")
ms2 = ms2.set_index("dataset.name")

In [ ]:
ms1.join(ms2, lsuffix="_l", rsuffix="_r")

In [ ]:
ms = ms1.join(ms2, lsuffix="_l", rsuffix="_r")

for measure in ["loss_train", "loss_test", "gradient_norm_test", "weights_product"]:
    grid = sns.relplot(
        data=ms,
        x="{}_l".format(measure),
        y="{}_r".format(measure),
    )
    ax = grid.axes[0][0]
    if "loss" in measure:
        ax.set_xscale("log")
        ax.set_yscale("log")
        lim = (
            min(ms["{}_l".format(measure)].min(), ms["{}_r".format(measure)].min()),
            max(ms["{}_l".format(measure)].max(), ms["{}_r".format(measure)].max()),
        )
        ax.set_xlim(lim)
        ax.set_ylim(lim)

    plt.show()

In [ ]:
ms["loss_train_l"]